<a href="https://colab.research.google.com/github/Sapphrain/OpenSourceDev_JiwooChoi/blob/main/Bike_Convenience_by_Busan_District.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install koreanize-matplotlib
import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.5 MB/s eta 0:00:00


In [24]:
import os
import pandas as pd
import json

# 데이터 디렉터리
data_dir = '/content/bike_data'
data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]

# CSV 병합 함수
def load_and_combine_data(files):
    all_data = []
    for f in files:
        try:
            # utf-8
            df = pd.read_csv(f, encoding='utf-8')
            all_data.append(df)
        except UnicodeDecodeError:
            try:
                # cp949
                df = pd.read_csv(f, encoding='cp949')
                all_data.append(df)
            except Exception as e:
                print(f"Failed to load {f}: {e}")
    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# 데이터 로딩
combined_df = load_and_combine_data(data_files)

# 구별 데이터 분석
if 'district' in combined_df.columns:
    district_counts = combined_df['district'].value_counts().reset_index()
    district_counts.columns = ['district', 'count']
else:
    district_counts = pd.DataFrame(columns=['district', 'count'])

# 위경도 추출
if 'latitude' in combined_df.columns and 'longitude' in combined_df.columns:
    marker_data = combined_df[['latitude', 'longitude']].dropna().to_dict('records')
else:
    marker_data = []

# HTML 문자열 생성
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>부산시 자전거 이용 분석 지도</title>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <style>#mapid {{ height: 600px; }}</style>
</head>
<body>

    <h1>부산시 자전거 이용 분석 지도</h1>
    <div id="mapid"></div>

    <h2>구별 자전거 관련 데이터 개수</h2>
    <div id="analysis-results"><p>Loading...</p></div>

    <script>
        const map = L.map('mapid').setView([35.1796, 129.0756], 12);

        const vworldApiKey = 'E0DE6BCD-5494-347B-B25B-2B60F252D4C8';
        const vworldUrl = "http://xdworld.vworld.kr:8080/2d/Base/202002/{{z}}/{{x}}/{{y}}.png?key=" + vworldApiKey;

        L.tileLayer(vworldUrl, {{
            maxZoom: 19,
            attribution: '&copy; <a href="http://www.vworld.kr/">VWorld</a>'
        }}).addTo(map);

        const markerData = {json.dumps(marker_data)};
        markerData.forEach(point => {{
            if (point.latitude && point.longitude) {{
                L.marker([point.latitude, point.longitude]).addTo(map)
                    .bindPopup('자전거 관련 위치');
            }}
        }});

        const analysisResults = {json.dumps(district_counts.to_dict('records'))};
        const analysisDiv = document.getElementById('analysis-results');
        analysisDiv.innerHTML = '<ul>';
        analysisResults.forEach(item => {{
            analysisDiv.innerHTML += '<li>' + item.district + ': ' + item.count + '개</li>';
        }});
        analysisDiv.innerHTML += '</ul>';
    </script>

</body>
</html>
"""

# HTML 저장
with open('bike_map.html', 'w', encoding='utf-8') as f:
    f.write(html_content)
